### 1. Lógica de NN para el 2 armed bandit

In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

# ======= 1. ENTORNO: RestlessBandit de 2 brazos =======
class RestlessBandit:
    def __init__(self, volatility=0.1):
        # Inicializamos p para el Brazo 0 entre 0.1 y 0.9.
        # El brazo 1 tendrá probabilidad = 1 - p.
        p = np.random.uniform(0.1, 0.9)
        self.probs = [p, 1 - p]
        self.volatility = volatility

    def pull(self, action):
        # Cada vez que se llama a pull, se actualizan las probabilidades:
        noise = np.random.randn() * self.volatility
        new_p = self.probs[0] + noise
        # Limitamos p entre 0.1 y 0.9 para evitar extremos.
        new_p = np.clip(new_p, 0.1, 0.9)
        self.probs[0] = new_p
        self.probs[1] = 1 - new_p

        # Retorna 1 con probabilidad de self.probs[action] o 0.
        return 1 if np.random.rand() < self.probs[action] else 0

### 2. Entrenamiento con PPO

In [57]:
# ======= 2. MODELO: Agente PPO con LSTM =======
class PPOAgent(nn.Module):
    def __init__(self, input_size=4, hidden_size=32, num_actions=2):
        """
        input_size: dimensión del vector de entrada. Se compone de:
                    - Acción previa en formato one-hot (2 dimensiones)
                    - Recompensa previa (1 dimensión)
                    - Timestep normalizado (1 dimensión)
                    Total: 2+1+1 = 4.
        """
        super(PPOAgent, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTMCell(input_size, hidden_size)
        self.policy_head = nn.Linear(hidden_size, num_actions)  # Produce los logits
        self.value_head = nn.Linear(hidden_size, 1)             # Estima el valor

    def reset_state(self):
        self.hx = torch.zeros(1, self.hidden_size)
        self.cx = torch.zeros(1, self.hidden_size)

    def forward(self, x):
        # x es de tamaño (1, input_size)
        self.hx, self.cx = self.lstm(x, (self.hx, self.cx))
        logits = self.policy_head(self.hx)
        value = self.value_head(self.hx)
        return logits, value

# ======= 3. Crear la entrada del agente =======
def get_input(last_action, last_reward, timestep, num_actions=2):
    """
    Construye un vector de entrada para el LSTM a partir de:
      - La acción previa (one-hot de dimensión 2)
      - La recompensa previa (dimensión 1)
      - El timestep normalizado (dimensión 1)
    """
    action_one_hot = F.one_hot(torch.tensor([last_action]), num_classes=num_actions).float()
    reward_tensor = torch.tensor([[last_reward]], dtype=torch.float32)
    timestep_tensor = torch.tensor([[timestep / 10.0]], dtype=torch.float32)  # Normalizamos para evitar valores altos
    x = torch.cat([action_one_hot, reward_tensor, timestep_tensor], dim=1)
    #concatena los tensores de manera horizontal (dim = 1)
    return x

# ======= 4. HIPERPARÁMETROS DE PPO =======
gamma = 0.99           # Factor de descuento
clip_epsilon = 0.2     # Parámetro de recorte PPO (clip)
ppo_epochs = 4         # Número de épocas por actualización
lr = 0.009             # Tasa de aprendizaje

agent = PPOAgent()
optimizer = optim.Adam(agent.parameters(), lr=lr)

num_episodes = 1000    # Cantidad total de episodios
episode_length = 5     # Pasos por episodio

# ======= 5. CICLO DE ENTRENAMIENTO CON PPO =======
for episode in range(num_episodes):
    # Usamos el entorno RestlessBandit con volatilidad definida (0.1 para probar en este caso)
    env = RestlessBandit(volatility=0.1)
    agent.reset_state()
    
    # Listas para almacenar la trayectoria del episodio
    states = []
    actions = []
    rewards = []
    log_probs = []
    values = []
    
    # Inicializamos: se parte de una acción por defecto (0) y recompensa 0 para el primer paso.
    last_action = 0
    last_reward = 0.0
    
    # Recorrido del episodio
    for t in range(episode_length):
        x = get_input(last_action, last_reward, t)
        logits, value = agent(x)
        probs = F.softmax(logits, dim=1)
        dist = torch.distributions.Categorical(probs)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        
        # Guardamos los datos de la transición
        states.append(x)
        actions.append(action)
        log_probs.append(log_prob)
        values.append(value)
        
        # Ejecutamos la acción en el entorno y obtenemos la recompensa.
        reward = env.pull(action.item())
        rewards.append(reward)
        
        last_action = action.item()
        last_reward = reward
    
    # ======= 5.1. Calcular los RETURNS y las VENTAJAS =======
    returns = []
    G = 0
    for r in reversed(rewards):
        G = r + gamma * G
        returns.insert(0, G)
    returns = torch.tensor(returns, dtype=torch.float32).unsqueeze(1)
    values = torch.cat(values)
    advantages = returns - values.detach()
    
    old_log_probs = torch.cat(log_probs).detach()
    
    # ======= 5.2. Actualización PPO sobre la trayectoria recogida =======
    for _ in range(ppo_epochs):
        new_log_probs = []
        new_values = []
        agent.reset_state()  # Reiniciamos el estado para reevaluar la trayectoria almacenada
        
        # Se reevalúa cada estado almacenado en la trayectoria
        for i, x in enumerate(states):
            logits, value = agent(x)
            probs = F.softmax(logits, dim=1)
            dist = torch.distributions.Categorical(probs)
            new_log_probs.append(dist.log_prob(actions[i]))
            new_values.append(value)
        new_log_probs = torch.cat(new_log_probs)
        new_values = torch.cat(new_values)
        
        ratio = torch.exp(new_log_probs - old_log_probs)
        surr1 = ratio * advantages
        surr2 = torch.clamp(ratio, 1 - clip_epsilon, 1 + clip_epsilon) * advantages
        policy_loss = -torch.min(surr1, surr2).mean()
        value_loss = F.mse_loss(new_values, returns)
        loss = policy_loss + 0.5 * value_loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (episode+1) % 100 == 0:
        total_reward = sum(rewards)
        print(f"Episode {episode+1}, Total Reward: {total_reward}, Loss: {loss.item():.4f}")

print("Entrenamiento completado.")


Episode 100, Total Reward: 4, Loss: -0.0631
Episode 200, Total Reward: 4, Loss: -0.1427
Episode 300, Total Reward: 2, Loss: 0.4094
Episode 400, Total Reward: 1, Loss: 1.9543
Episode 500, Total Reward: 4, Loss: -0.3483
Episode 600, Total Reward: 4, Loss: 0.1085
Episode 700, Total Reward: 2, Loss: -0.0126
Episode 800, Total Reward: 3, Loss: -0.1049
Episode 900, Total Reward: 3, Loss: 1.0354
Episode 1000, Total Reward: 3, Loss: -0.3200
Entrenamiento completado.


# 3. Ejemplo de funcionamiento con un agente en modo evaluación

In [62]:
# ======= 6. Evaluación del agente entrenado con PPO =======
agent.eval()  # Cambia el modelo a modo evaluación
env = RestlessBandit(volatility=0.1)  # Nuevo episodio de prueba en entorno cambiante
agent.reset_state()

last_action = 0
last_reward = 0
total_reward = 0

print("Probabilidades ocultas del entorno:", env.probs)

for t in range(5):  # Evaluamos por 5 pasos
    with torch.no_grad():
        x = get_input(last_action, last_reward, t)
        logits, _ = agent(x)
        probs = F.softmax(logits, dim=1)
        action = torch.multinomial(probs, num_samples=1).item()

    reward = env.pull(action)
    total_reward += reward
    print(f"Paso {t} | Acción: {action} | Recompensa: {reward}")
    last_action = action
    last_reward = reward

print("Recompensa total:", total_reward)

Probabilidades ocultas del entorno: [0.6392585397974725, 0.3607414602025275]
Paso 0 | Acción: 0 | Recompensa: 1
Paso 1 | Acción: 0 | Recompensa: 1
Paso 2 | Acción: 0 | Recompensa: 0
Paso 3 | Acción: 0 | Recompensa: 1
Paso 4 | Acción: 0 | Recompensa: 1
Recompensa total: 4
